# Data management

### Introduction to Text Data

## [Malka Guillot](https://malkaguillot.github.io/)

### HEC Liège | [ECON2306]()

This notebook provides an introduction to the basic tools for text analytics.

### Set up environment

In [1]:
#!pip install unidecode
#!pip install googletrans
#!pip install gensim
#!pip install spacy
#!pip install wordcloud
#!pip install pyldavis

#!python -m spacy download en_core_web_sm
#!python -m spacy download en_core_web_lg

import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
# Common imports
import numpy as np
import os
import pandas as pd

# To plot pretty figures
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib as mpl
import matplotlib.pyplot as plt
#%matplotlib notebook
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import seaborn as sns
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings = lambda *a, **kw: None

# to make this notebook's output identical at every run
np.random.seed(42)

In [3]:
# Scikit-Learn ≥0.20 is required
import sklearn

## Set up the data

We use as an example the **20 Newsgroups** ([[http://qwone.com/~jason/20Newsgroups/]]) dataset (from `sklearn`), a collection of about 20,000 newsgroup (message forum) documents.

In [4]:
from sklearn.datasets import fetch_20newsgroups
data = fetch_20newsgroups() # object is a dictionary
data.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

Data Set Characteristics:

In [5]:
#print(data['DESCR'])

In [6]:
W, y = data.data, data.target
n_samples = y.shape[0]
n_samples

11314

In [7]:
y[:10] # news story categories

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

`W` is  list of documents:

In [8]:
doc = W[0]
doc

"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"

**Make a pandas dataframe**

In [9]:
df = pd.DataFrame(W,columns=['text'])
df['topic'] = y
df.head()

,text,topic
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14


## Dictionary Methods

### Sentiment Analysis

In [10]:
#!pip install nltk
import nltk

# Download the lexicon
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [11]:
# Import the lexicon
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Create an instance of SentimentIntensityAnalyzer
sent_analyzer = SentimentIntensityAnalyzer()

# Example
sentence = "VADER is pretty good at identifying the underlying sentiment of a text!"
print(sent_analyzer.polarity_scores(sentence))

{'neg': 0.0, 'neu': 0.585, 'pos': 0.415, 'compound': 0.75}


In [12]:
# Dictionary-Based Sentiment Analysis

sid = SentimentIntensityAnalyzer()
polarity = sid.polarity_scores(doc)
print(polarity)

{'neg': 0.012, 'neu': 0.916, 'pos': 0.072, 'compound': 0.807}


In [13]:
doc

"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"

The text transformation is often applied to a dataframe column:

In [14]:
# sample 20% of the dataset
dfs = df.sample(frac=.2)

# apply compound sentiment score to data-frame
def get_sentiment(snippet):
    return sid.polarity_scores(snippet)['compound']
dfs['sentiment'] = dfs['text'].apply(get_sentiment)

In [15]:
dfs.sort_values('sentiment',inplace=True)
# print beginning of most positive documents
[x[60:150] for x  in dfs[-5:]['text']]

['CLINTON: AM Press Briefing by Dee Dee Myers -- 4.15.93\nOrganization: Project GNU, Free Sof',
 ' Newsletter, Part 2/4\nReply-To: david@stat.com (David Dodell)\nDistribution: world\nOrganiza',
 "CLINTON: President's Remarks at Town Hall Meeting\nOrganization: MIT Artificial Intelligenc",
 'Final Public Dragon Magazine Update (Last chance for public bids)\nKeywords: Dragon Magazin',
 'CLINTON: Background BRiefing in Vancouver 4.4.93\nOrganization: Project GNU, Free Software ']

In [16]:
# print beginning of most negative documents
[x[50:150] for x  in dfs[:5]['text']]

['OSE OF FALKLANDS WAR\nOrganization: University of Wisconsin Eau Claire\nLines: 918\n\n\n\nSECRET PURPOSE O',
 'ct: Re: Need info on 43:1 and suicide for refutation\nOrganization: Ideology Busters, Inc.\nDistributi',
 'ews in LATVIA - some documents\nArticle-I.D.: zuma.9304052018\nReply-To: sera@zuma.UUCP (Serdar Argic)',
 ': Accounts of Anti-Armenian Human Rights Violations in Azerbaijan #013\nSummary: Prelude to Current E',
 'year the Turkish Nation is mourning and praying again for...\nReply-To: sera@zuma.UUCP (Serdar Argic)']

### Dictionary specific to a corpus
Counting sets of words or phrases across documents


In [17]:
from collections import Counter
freqs = Counter()
for i, row in df.iterrows():
    freqs.update(row['text'].lower().split())
    if i > 100:
        break
freqs.most_common()[:20] # can use most frequent words as style/function words

[('the', 1357),
 ('of', 645),
 ('to', 578),
 ('and', 548),
 ('a', 532),
 ('in', 427),
 ('i', 370),
 ('is', 360),
 ('that', 300),
 ('for', 250),
 ('it', 193),
 ('on', 189),
 ('you', 173),
 ('have', 161),
 ('be', 158),
 ('-', 156),
 ('with', 156),
 ('this', 148),
 ('are', 142),
 ('>', 140)]

### Removing words from a curpus vocabulary: StopWords

In [18]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
# Print the first 10 stopwords
list(stopwords)[:10]

['ma',
 'under',
 'a',
 'mightn',
 'shouldn',
 "weren't",
 'y',
 's',
 'are',
 'ourselves']

In [20]:
stopwords = set([x for x in stopwords if x != 'against']) # add an exception

In [21]:
stopfreq = np.sum([freqs[x] for x in stopwords]) #
stopfreq

10481

In [22]:
otherfreq = np.sum([freqs[x] for x in freqs if x not in stopwords])
otherfreq

18320

## RegEx

Please refer to [RegExOne Regular Expressions Lessons](regexone.com) and [the python documentation](https://docs.python.org/3/howto/regex.html).

In [23]:
import re

docs = dfs[:5]['text']

# Extract words after Subject.
for doc in docs:
    print(re.findall(r'Subject: \w+ ', # pattern to match. always put 'r' in front of string so that backslashes are treated literally.
                     doc,            # string
                     re.IGNORECASE))  # ignore upper/lowercase (optional)

['Subject: SECRET ']
['Subject: Gun ']
[]
['Subject: Accounts ']
['Subject: This ']


In [24]:
# Extract hyphenated words
for doc in docs:
    print(re.findall(r'[a-z]+-[a-z]+',
                     doc,
                     re.IGNORECASE))

['IN-VISIBILITY', 'wind-swept', 'so-called', 'longer-range', 'so-called', 'two-thirds', 'semi-permanent', 'far-off', 'so-called', 'fast-approaching', 'so-called', 'up-to', 'anti-Bolshevik', 'long-range', 'two-prong', 'Rockefeller-controlled', 'full-fledged', 'all-out', 'so-called', 'then-President', 'over-all', 'so-called', 'particle-beam', 'man-made', 'then-Secretary', 'newly-launched', 'nuclear-powered', 'so-called', 'war-planes', 'Stealth-field', 'three-phase', 'war-room', 'all-out', 'anti-Bolshevik', 'anti-Bolshevik', 'so-called', 'slow-down', 'anti-Bolshevik', 'Rockefeller-Russian', 'mid-February', 'Bolshevik-controlled', 'long-standing', 'scrap-metal', 'work-site', 'year-old', 'long-standing', 'far-off', 'Rockefeller-Russian', 'high-speed', 'Rockefeller-Russian', 'Rockefeller-Russian', 'so-called', 'blast-proof', 'Rockefeller-Russian', 'mid-Atlantic', 'Rockefeller-Russian', 'hollowed-out', 'beam-weapon', 'Range-Finding', 'non-homing', 'non-nuclear', 'pseudo-smile', 'Rockefeller-R

In [25]:
# extract email addresses
for i, doc in enumerate(docs):
    finder = re.finditer('\w+@.+\.\w\w\w', # pattern to match ([^\s] means non-white-space)
                     doc)            # string
    for m in finder:
        print(i, m.span(),m.group()) # location (start,end) and matching string

0 (6, 29) mcelwre@cnsvax.uwec.edu
0 (274, 291) eidetics@cerf.net
1 (6, 27) lvc@cbnews.cb.att.com
1 (11768, 11784) cipriani@att.com
2 (6, 19) sera@zuma.UUC
2 (121, 134) sera@zuma.UUC
2 (202, 249) 50D@unix.amherst.edu> nwbernst@unix.amherst.edu
3 (6, 25) dbd@urartu.sdpa.org
3 (21578, 21597) dbd@urartu.sdpa.org
4 (6, 19) sera@zuma.UUC
4 (121, 134) sera@zuma.UUC


In [26]:
# baker-bloom economic uncertainty
pattern1 = r'(\b)uncertain[a-z]*'
pattern2 = r'(\b)econom[a-z]*'
pattern3 = r'(\b)congress(\b)|(\b)deficit(\b)|(\b)federal reserve(\b)|(\b)legislation(\b)|(\b)regulation(\b)|(\b)white house(\b)'

In [27]:
re.search(pattern1,'The White House tried to calm uncertainty in the markets.')

<re.Match object; span=(30, 41), match='uncertainty'>

In [28]:
def indicates_uncertainty(doc):
    m1 = re.search(pattern1, doc, re.IGNORECASE)
    m2 = re.search(pattern2, doc, re.IGNORECASE)
    m3 = re.search(pattern3, doc, re.IGNORECASE)
    if m1 and m2 and m3:
        return True
    else:
        return False

In [29]:
df['uncertainty'] = df['text'].apply(indicates_uncertainty)

In [30]:
df.uncertainty.mean()

0.00017677214071062401

In [31]:
df[df.uncertainty]

,text,topic,uncertainty
2350,From: ld231782@longs.lance.colostate.edu (L. D...,11,True
4682,From: ld231782@longs.lance.colostate.edu (L. D...,11,True


## A standard tokenization pipeline


In [32]:
text = "Marie Curie was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in 2 scientific fields. Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first married couple to win the Nobel Prize and launching the Curie family legacy of 5 Nobel Prizes."

### 1. Pre-processing text

#### Removing capitalization, punctuation,  numbers

In [33]:
# Capitalization
text_lower = text.lower() # go to lower-case
text_lower

'marie curie was the first woman to win a nobel prize, the first person to win a nobel prize twice, and the only person to win a nobel prize in 2 scientific fields. her husband, pierre curie, was a co-winner of her first nobel prize, making them the first married couple to win the nobel prize and launching the curie family legacy of 5 nobel prizes.'

In [34]:
# Punctuation

# recipe for fast punctuation removal
from string import punctuation
punc_remover = str.maketrans('','',punctuation)
text_nopunc = text_lower.translate(punc_remover)
print(text_nopunc)

marie curie was the first woman to win a nobel prize the first person to win a nobel prize twice and the only person to win a nobel prize in 2 scientific fields her husband pierre curie was a cowinner of her first nobel prize making them the first married couple to win the nobel prize and launching the curie family legacy of 5 nobel prizes


#### Tokenization

In [35]:
# Tokens
tokens = text_nopunc.split() # splits a string on white space
print(tokens)

['marie', 'curie', 'was', 'the', 'first', 'woman', 'to', 'win', 'a', 'nobel', 'prize', 'the', 'first', 'person', 'to', 'win', 'a', 'nobel', 'prize', 'twice', 'and', 'the', 'only', 'person', 'to', 'win', 'a', 'nobel', 'prize', 'in', '2', 'scientific', 'fields', 'her', 'husband', 'pierre', 'curie', 'was', 'a', 'cowinner', 'of', 'her', 'first', 'nobel', 'prize', 'making', 'them', 'the', 'first', 'married', 'couple', 'to', 'win', 'the', 'nobel', 'prize', 'and', 'launching', 'the', 'curie', 'family', 'legacy', 'of', '5', 'nobel', 'prizes']


In [36]:
# Numbers
# remove numbers (keep if not a digit)
no_numbers = [t for t in tokens if not t.isdigit()]
# keep if not a digit, else replace with "#"
norm_numbers = [t if not t.isdigit() else '#'
                for t in tokens ]
print(no_numbers )
print(norm_numbers)

['marie', 'curie', 'was', 'the', 'first', 'woman', 'to', 'win', 'a', 'nobel', 'prize', 'the', 'first', 'person', 'to', 'win', 'a', 'nobel', 'prize', 'twice', 'and', 'the', 'only', 'person', 'to', 'win', 'a', 'nobel', 'prize', 'in', 'scientific', 'fields', 'her', 'husband', 'pierre', 'curie', 'was', 'a', 'cowinner', 'of', 'her', 'first', 'nobel', 'prize', 'making', 'them', 'the', 'first', 'married', 'couple', 'to', 'win', 'the', 'nobel', 'prize', 'and', 'launching', 'the', 'curie', 'family', 'legacy', 'of', 'nobel', 'prizes']
['marie', 'curie', 'was', 'the', 'first', 'woman', 'to', 'win', 'a', 'nobel', 'prize', 'the', 'first', 'person', 'to', 'win', 'a', 'nobel', 'prize', 'twice', 'and', 'the', 'only', 'person', 'to', 'win', 'a', 'nobel', 'prize', 'in', '#', 'scientific', 'fields', 'her', 'husband', 'pierre', 'curie', 'was', 'a', 'cowinner', 'of', 'her', 'first', 'nobel', 'prize', 'making', 'them', 'the', 'first', 'married', 'couple', 'to', 'win', 'the', 'nobel', 'prize', 'and', 'launch

In [37]:
# Stopwords
from nltk.corpus import stopwords
stoplist = stopwords.words('english')
# keep if not a stopword
nostop = [t for t in norm_numbers if t not in stoplist]
print(nostop)

['marie', 'curie', 'first', 'woman', 'win', 'nobel', 'prize', 'first', 'person', 'win', 'nobel', 'prize', 'twice', 'person', 'win', 'nobel', 'prize', '#', 'scientific', 'fields', 'husband', 'pierre', 'curie', 'cowinner', 'first', 'nobel', 'prize', 'making', 'first', 'married', 'couple', 'win', 'nobel', 'prize', 'launching', 'curie', 'family', 'legacy', '#', 'nobel', 'prizes']


In [38]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [39]:
# spacy stopwords
sorted(list(nlp.Defaults.stop_words))[:10]

["'d", "'ll", "'m", "'re", "'s", "'ve", 'a', 'about', 'above', 'across']

#### Stemming & Lemmatizing

In [40]:
# Stemming
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('english') # snowball stemmer, english
# remake list of tokens, replace with stemmed versions
tokens_stemmed = [stemmer.stem(t) for t in ['tax','taxes','taxed','taxation']]
print(tokens_stemmed)

['tax', 'tax', 'tax', 'taxat']


In [41]:
stemmer = SnowballStemmer('german') # snowball stemmer, german
print(stemmer.stem("Autobahnen"))

autobahn


In [42]:
# Lemmatizing
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
[wnl.lemmatize(c) for c in ['corporation', 'corporations', 'corporate']]

['corporation', 'corporation', 'corporate']

#### Pre-processing function (homemade)

In [43]:
from string import punctuation
translator = str.maketrans('','',punctuation)
from nltk.corpus import stopwords
stoplist = set(stopwords.words('english'))
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('english')

def normalize_text(doc):
    "Input doc and return clean list of tokens"
    doc = doc.replace('\r', ' ').replace('\n', ' ')
    lower = doc.lower() # all lower case
    nopunc = lower.translate(translator) # remove punctuation
    words = nopunc.split() # split into tokens
    nostop = [w for w in words if w not in stoplist] # remove stopwords
    no_numbers = [w if not w.isdigit() else '#' for w in nostop] # normalize numbers
    stemmed = [stemmer.stem(w) for w in no_numbers] # stem each word
    return stemmed

And apply it to the corpus.

In [44]:
df['tokens_cleaned'] = df['text'].apply(normalize_text)
df['tokens_cleaned']

,tokens_cleaned
0,"[lerxstwamumdedu, where, thing, subject, car, ..."
1,"[guykuocarsonuwashingtonedu, guy, kuo, subject..."
2,"[twillisececnpurdueedu, thoma, e, willi, subje..."
3,"[jgreenamb, joe, green, subject, weitek, p9000..."
4,"[jcmheadcfaharvardedu, jonathan, mcdowel, subj..."
...,...
11309,"[jimzisfeinfactorycom, jim, zisfein, subject, ..."
11310,"[ebodinpearltuftsedu, subject, screen, death, ..."
11311,"[westesnetcomcom, est, subject, mount, cpu, co..."
11312,"[stevehcrlgw, steven, collin, subject, sphere,..."


#### Pre-processing function (readymade)
**Shortcut: `gensim.simple_preprocess`.**

In [45]:
from gensim.utils import simple_preprocess
print(simple_preprocess(text))

['marie', 'curie', 'was', 'the', 'first', 'woman', 'to', 'win', 'nobel', 'prize', 'the', 'first', 'person', 'to', 'win', 'nobel', 'prize', 'twice', 'and', 'the', 'only', 'person', 'to', 'win', 'nobel', 'prize', 'in', 'scientific', 'fields', 'her', 'husband', 'pierre', 'curie', 'was', 'co', 'winner', 'of', 'her', 'first', 'nobel', 'prize', 'making', 'them', 'the', 'first', 'married', 'couple', 'to', 'win', 'the', 'nobel', 'prize', 'and', 'launching', 'the', 'curie', 'family', 'legacy', 'of', 'nobel', 'prizes']


In [46]:
from collections import Counter
print(Counter(simple_preprocess(text)))

Counter({'the': 6, 'nobel': 6, 'prize': 5, 'first': 4, 'to': 4, 'win': 4, 'curie': 3, 'was': 2, 'person': 2, 'and': 2, 'her': 2, 'of': 2, 'marie': 1, 'woman': 1, 'twice': 1, 'only': 1, 'in': 1, 'scientific': 1, 'fields': 1, 'husband': 1, 'pierre': 1, 'co': 1, 'winner': 1, 'making': 1, 'them': 1, 'married': 1, 'couple': 1, 'launching': 1, 'family': 1, 'legacy': 1, 'prizes': 1})


Now let's `simple_preprocess` on the corpus.

In [47]:
df['tokens_simple'] = df['text'].apply(simple_preprocess)
df['tokens_simple']

,tokens_simple
0,"[from, lerxst, wam, umd, edu, where, my, thing..."
1,"[from, guykuo, carson, washington, edu, guy, k..."
2,"[from, twillis, ec, ecn, purdue, edu, thomas, ..."
3,"[from, jgreen, amber, joe, green, subject, re,..."
4,"[from, jcm, head, cfa, harvard, edu, jonathan,..."
...,...
11309,"[from, jim, zisfein, factory, com, jim, zisfei..."
11310,"[from, ebodin, pearl, tufts, edu, subject, scr..."
11311,"[from, westes, netcom, com, will, estes, subje..."
11312,"[from, steve, hcrlgw, steven, collins, subject..."


### 2. Count and frequencies

In [48]:
# Counter is a quick pure-python solution.
from collections import Counter
freqs = Counter(tokens)
freqs.most_common()[:20]

[('the', 6),
 ('nobel', 6),
 ('prize', 5),
 ('first', 4),
 ('to', 4),
 ('win', 4),
 ('a', 4),
 ('curie', 3),
 ('was', 2),
 ('person', 2),
 ('and', 2),
 ('her', 2),
 ('of', 2),
 ('marie', 1),
 ('woman', 1),
 ('twice', 1),
 ('only', 1),
 ('in', 1),
 ('2', 1),
 ('scientific', 1)]

Usually we use scikit-learn's vectorizer.

In [49]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(min_df=0.001, # at min 0.1% of docs
                        max_df=.8, # drop if shows up ih more than 80%
                        max_features=1000,
                        stop_words='english',
                        ngram_range=(1,3)) # words, bigrams, and trigrams
X = vec.fit_transform(df['text'])

# save the vectors
# pd.to_pickle(X,'X.pkl')

# save the vectorizer
# (so you can transform other documents,
# also for the vocab)
#pd.to_pickle(vec, 'vec-3grams-1.pkl')

In [50]:
X

<11314x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 526707 stored elements in Compressed Sparse Row format>

In [51]:
# tf-idf vectorizer up-weights rare/distinctive words
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(min_df=0.001,
                        max_df=0.9,
                        max_features=1000,
                        stop_words='english',
                        use_idf=True, # the new piece
                        ngram_range=(1,2))

X_tfidf = tfidf.fit_transform(df['text'])
#pd.to_pickle(X_tfidf,'X_tfidf.pkl')

In [52]:
X_tfidf

<11314x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 521387 stored elements in Compressed Sparse Row format>

#### 3. N-grams

In [53]:
from nltk import ngrams
from collections import Counter

# get n-gram counts for 10 documents
grams = []
for i, row in df.iterrows():
    tokens = row['text'].lower().split() # get tokens
    for n in range(2,4):
        grams += list(ngrams(tokens,n)) # get bigrams, trigrams, and quadgrams
    if i > 50:
        break
Counter(grams).most_common()[:8]  # most frequent n-grams

[(('of', 'the'), 41),
 (('subject:', 're:'), 37),
 (('in', 'the'), 33),
 (('to', 'the'), 27),
 (('i', 'am'), 21),
 (('i', 'have'), 21),
 (('to', 'be'), 19),
 (('on', 'the'), 18)]

In [54]:
# spacy NER noun chunks
i = 0
chunks = list(nlp(df['text'].iloc[10]).noun_chunks)
chunks

[(Irwin Arnstein,
 Subject,
 Duc
 Summary,
 What,
 it,
 Distribution,
 Sat,
 1 May 1993 05:00:00 GMT
 Organization,
 CompuTrac Inc.,
 Richardson TX
 Keywords,
 Ducati,
 GTS,
 13
 
 I,
 a line,
 a Ducati 900GTS 1978 model,
 17k,
 the clock,
 paint,
 the bronze/brown/orange,
 a bit,
 oil,
 1st,
 hard accel,
 The shop,
 trans,
 oil 
 leak,
 They,
 the bike,
 the 1 and only owner,
 They,
 I,
 Any opinions,
 me,
 Thanks,
 It,
 a nice stable mate,
 the Beemer,
 I,
 a jap bike,
 myself,
 Axis Motors,
 Irwin,
 I,
 I,
 CompuTrac-Richardson,Tx
 irwin@cmptrc.lonestar.org    DoD,
 (R75/6]

#### 4. Parts of speech (and Corpus Prep with spaCy)

Get spacy documents for each speech and add to dataframe. This is quicker than iterating over the dataframe with `iterrows()`, but slower than a parallelized solution. It will take a few minutes for a whole corpus.

In [55]:
dfs = df.sample(10)
dfs['doc'] = dfs['text'].apply(nlp)

In [56]:
dfs[['text','doc']].head(2)

,text,doc
6889,From: wcd82671@uxa.cso.uiuc.edu (daniel warren...,"(From, :, wcd82671@uxa.cso.uiuc.edu, (, daniel..."
9583,From: dxf12@po.cwru.edu (Douglas Fowler)\nSubj...,"(From, :, dxf12@po.cwru.edu, (, Douglas, Fowle..."


In [57]:
# The spacy model already gives you sentences and tokens.
# For example:
tensents = list(dfs['doc'].iloc[0].sents)[:10] #
tensents

[From: wcd82671@uxa.cso.uiuc.edu (daniel warren c)
 Subject: Hard Copy --- Hot Pursuit!!!!,
 Summary: SHIT!!!!!!!,
 Keywords: Running from the Police.
 Article-I.D.: news.,
 C5J34y.2t4
 Distribution: rec.motorcycles
 Organization: University of Illinois at Urbana
 Lines: 44
 
 
 Yo, did anybody see this run of HARD COPY?
 ,
 I guy on a 600 Katana got pulled over by the Police (I guess for
 speeding or something).  ,
 But just as the cop was about to step
 out of the car, the dude punches it down an interstate in Georgia.,
 Ang then, the cop gives chase.
 ,
 Now this was an interesting episode because it was all videotaped!!!,
 Everything from the dramatic takeoff and 135mph chase to the sidestreet
 battle at about 100mph.  ,
 What happened at the end?  ]

In [58]:
# tokens
list(tensents[-1])

[What, happened, at, the, end, ?,  ]

In [59]:
# lemmas
[x.lemma_ for x in tensents[-1]]

['what', 'happen', 'at', 'the', 'end', '?', ' ']

In [60]:
# POS tags
[x.tag_ for x in tensents[-1]]

['WP', 'VBD', 'IN', 'DT', 'NN', '.', '_SP']

## Document Distance

In [61]:
# compute pair-wise similarities between all documents in corpus"
from sklearn.metrics.pairwise import cosine_similarity

sim = cosine_similarity(X[:100])
sim.shape

(100, 100)

In [62]:
sim[:4,:4]

array([[1.        , 0.20384233, 0.15095711, 0.19219753],
       [0.20384233, 1.        , 0.12569587, 0.1608558 ],
       [0.15095711, 0.12569587, 1.        , 0.16531366],
       [0.19219753, 0.1608558 , 0.16531366, 1.        ]])

In [63]:
# TF-IDF Similarity
tsim = cosine_similarity(X_tfidf[:100])
tsim[:4,:4]

array([[1.        , 0.05129256, 0.08901433, 0.06064389],
       [0.05129256, 1.        , 0.07497709, 0.03570566],
       [0.08901433, 0.07497709, 1.        , 0.09077347],
       [0.06064389, 0.03570566, 0.09077347, 1.        ]])